###  Introduction 
Different dynamics have contriuted to continuous evolution of the entertainment industry in recent years. For instance, technology has played a critical role in meeting the high  demand for quality and original video contents. This reality has prompted many movie companies to reinvent themseves in the production, capture and share of quality products to the market. Our company is not left behind on these emerging trends. We are setting up a new movie studio with an objective of capturing the market expectations by producing quality and original  films.  

To succeed in this, just like in any other venjure, we need a workig strategic plan and a comprehensive understanding knowledge of the market trends. First, we need to determine the types of films that are currenly dominating the market and analyze key characteristics and factors that contribute to commercial success.

Our project will explore the major successful films in the industry today and extract actionable insights to inform our movie studio's content strategy. By aligning our creative vision with the market's proven preferences, we aim to maximize our chances of producing hit films that capture the imagination of audiences worldwide.

## Objectives

### 1. What are the audiences preferrences?

### 2. Which movies are most profitable in terms of revenue and profit margin?

### 3. Which movies had highest ratings?

### 4.How movie release dates vary by month and determine peak release times e.g school holidays, Christmas, Valentines

### 5. How has the average box office revenue changed over time to identify trends that could impact movie production and marketing strategies?

### 6. Which Production Studios and Companies Are Dominating the Box Office?

### 7. Character popularity and group cast impact.Assess the financial impact of major characters and group casts in movies